In [ ]:
import os
import numpy as np
import pandas as pd
import spotipy
from py2opt.routefinder import RouteFinder

# travelling salesman
We want to find an path between tracks which minimises the distance of each step, visiting every track once and only once. This is essentially the [travelling salesman problem](https://en.m.wikipedia.org/wiki/Travelling_salesman_problem), which can be solved approximately using the [2-opt algorithm](https://en.m.wikipedia.org/wiki/2-opt).

First, we'll load in the data we've previously collected.

In [ ]:
playlist_id = os.environ["PLAYLIST_ID"]
distance_matrix = np.load(f"./data/{playlist_id}_distance.npy")

In [ ]:
df = pd.read_csv(
    f"./data/{playlist_id}_features.csv",
    index_col=0
)

Now we can run py2opt's version of the 2-opt algorithm to find an idealised path through all the nodes in our graph/distance matrix.

`py2opt` is a very chatty library, so we'll also use a bit of ipython magic to [capture](https://ipython.readthedocs.io/en/stable/interactive/magics.html#cellmagic-capture) and discard the output of the function - we'll still retain the meaningful results.

In [ ]:
%%capture 

route_finder = RouteFinder(
    distance_matrix.tolist(), 
    df.index.values.tolist(), 
    iterations=100,

)

best_distance, best_route = route_finder.solve()

In [ ]:
best_distance

That's it, we have ain idealised path through our tracks according to their features!

In [ ]:
sp = spotipy.Spotify(
    client_credentials_manager=spotipy.oauth2.SpotifyClientCredentials()
)

for track_id in best_route:
    track = sp.track(track_id)
    print(f"{track['artists'][0]['name']}\n  {track['name']}\n")